<a href="https://colab.research.google.com/github/SG-Akshay10/Dynamic_Programming/blob/main/PSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Particle Swarm Optimization**

### **Question**: 
Design a neural network (the choice of implementation model can be pytorch, tensorflow or the whitebox model) for the data set shared in the ML lab assignment for neural networks. 

* Develop individual code base using following algorithms for weight optimization:
1.	Genetic Algorithm
2.	Cultural Algorithm
3.	Particle Swarm Optimization
4.	Ant Colony Optimization
* 
Data to be uploaded to github
1.	Note on the comparison of performance for the four methods. 
2.	The codebase for all four methods 
3.	The research papers that you have referred to.



### Dependencies and Dataset

In [103]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [104]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.utils import to_categorical

In [105]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [106]:
import os
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import datasets, transforms

In [107]:
df= pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Bank_Personal_Loan_Modelling.csv")

Columns of the dataset : 
* ID: Customer ID
* Age: Customer Age
* Experience: Amount of work experience in years
* Income: Amount of annual income (in thousands)
* Zipcode: Zipcode of where customer lives
* Family: Number of family members
* CCAvg: Average monthly credit card spendings
* Education: Education level (1: Bachelor, 2: Master, 3: Advanced Degree)
* Mortgage: Mortgage of house (in thousands)
* Securities Account: Boolean of whether customer has a securities account
* CD Account: Boolean of whether customer has Certificate of Deposit account
* Online: Boolean of whether customer uses online banking
* CreditCard: Does the customer use credit card issued by the bank?
* Personal Loan: This is the target variable (Binary Classification Problem)

#### Data Cleaning and Feature Engineering

In [108]:
# Deleting Columns which are not necessary
df.drop(["ID"],axis=1,inplace=True)

## Train Test Split

In [109]:
df.shape

(5000, 13)

In [110]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [111]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=69)

In [112]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [113]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4000, 12), (1000, 12), (4000,), (1000,))

# PyTorch 

In [114]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [115]:
BATCH_SIZE = 1

In [116]:
train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)

In [117]:
train_x.shape, train_y.shape

(torch.Size([4000, 12]), torch.Size([4000]))

In [118]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

## Building Model

In [119]:
class NeuralNetwork(torch.nn.Module):
    
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        
        self.layer1 = torch.nn.Linear(12,16)
        self.layer2 = torch.nn.Linear(16,8)
        self.layer3 = torch.nn.Linear(8,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        x = self.sigmoid(x)
        return x

# Weight Optimization using Particle Swarm Optimization

In [120]:
#pip install pyswarms

In [121]:
model = NeuralNetwork()
torch.set_grad_enabled(False)
param = np.concatenate([i.numpy().flatten() for i in model.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]

print("Dim : ", len(param))
print("Layers Shape : ", shape)
print("Layers Size : ", size)

Dim :  353
Layers Shape :  [(16, 12), (16,), (8, 16), (8,), (1, 8), (1,)]
Layers Size :  [192, 16, 128, 8, 8, 1]


In [122]:
def objective_function(particle, model, train_x, train_y,shape = shape,size=size):
    # Reshape the vector to weights and biases dimension
    accuracy = []
    output = []

    for par in particle:
      param = list()
      cum_sum = 0
      for i in range(len(size)):
          array = particle[cum_sum : cum_sum + size[i]]
          array = array.reshape(shape[i])
          cum_sum += size[i]
          param.append(array)
      param = np.array(param, dtype="object")
      output.append(param)
    
    for i in range(len(output)):
        # Copy Weights and Biases
      model = NeuralNetwork()
      for idx, wei in enumerate(model.parameters()):
          wei.data = (torch.tensor(param[idx])).to(torch.float32)

    # Calculate Accuracy
    y_pred = model(train_x)
    y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
    acc = (y_pred == train_y).sum().float().item() / len(train_x)
    accuracy = 1 - acc # Optimization function aims to reduce the cost so (1 - accuracy) 
    return accuracy

In [123]:
# Initialize the particles
n_particles = 50
dimensions = len(param)
x_max = 1.0 * np.ones(dimensions)
x_min = -1.0 * x_max
bounds = (x_max,x_min)
particles = np.random.uniform(low=x_min, high=x_max, size=(n_particles, dimensions))
best_fitness = np.inf
best_particle = None

# Initialize the velocities
v_max = 0.1 * (x_max - x_min)
v_min = -v_max
velocities = np.random.uniform(low=v_min, high=v_max, size=(n_particles, dimensions))

# Set the PSO parameters
c1 = 0.6
c2 = 0.3
w = 0.1

# Run the PSO algorithm
max_iter = 10
for i in range(max_iter):
    # Evaluate the fitness of the particles
    fitness = np.array([objective_function(particles[j], model, train_x, train_y) for j in range(n_particles)])

    # Update the global best
    if np.min(fitness) < best_fitness:
        best_fitness = np.min(fitness)
        best_particle = particles[np.argmin(fitness)]

    # Update the velocities and positions of the particles
    r1 = np.random.uniform(size=(n_particles, dimensions))
    r2 = np.random.uniform(size=(n_particles, dimensions))
    vel = w * velocities + c1 * r1 * (particles - particles) + c2 * r2 * (best_particle - particles)
    particles += vel

    # Clip the positions to the bounds
    particles = np.clip(particles, x_min, x_max)

# Extract best_cost and best_parameter
best_cost = best_fitness
best_parameter = best_particle

# Evaluate the performance of the best parameter
best_accuracy = 1 - objective_function(best_parameter, model, train_x, train_y)
print(f"Best Cost : {best_cost}\n Best Parameter : {best_parameter}")
print(f"Best Parameter : {best_accuracy}")

Best Cost : 0.09075
 Best Parameter : [-0.59401019 -0.01699175  0.18700409  0.38477674 -0.04957273  0.30518702
 -0.34307036  0.01512405 -0.487528   -0.59442507 -0.57619161 -0.75362338
 -0.20619303 -0.16755069 -0.26370085  0.14251865  0.08035043 -0.41555627
  0.80449172 -0.09222494 -0.6414535  -0.36190503  0.36468796  0.13265524
 -0.04739157 -0.17797701  0.30533657 -0.13779736 -0.19587841 -0.21033555
 -0.04767049 -0.02743238  0.01552382 -0.06655173 -0.43416052 -0.47661687
 -0.40128995 -0.00513083  0.55570843 -0.80153487  0.01156546  0.1365449
 -0.36535881 -0.31659334  0.3180034  -0.16857381 -0.49866742  0.10708761
 -0.62695691  0.96670232  0.4625531  -0.56669208  0.14303327  0.54627003
  0.40771674  0.83249649 -0.66537521 -0.3571346   0.12324702 -0.41875508
 -0.36064119 -0.22199844 -0.19008116  0.61051555 -0.41088959 -0.48267637
  0.08489716  0.05778405  0.4570001   0.0232404   0.44727717  0.62267433
  0.43953895  0.13042214 -0.13898829  0.09998575 -0.18039817  0.81614313
  0.12990161 -

## Testing 

In [124]:
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)

In [125]:
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=1)

### Classification Report

In [126]:
best_model = NeuralNetwork()

In [127]:
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))

              precision    recall  f1-score   support

         0.0       1.00      0.91      0.95      1000
         1.0       0.00      0.00      0.00         0

    accuracy                           0.91      1000
   macro avg       0.50      0.45      0.48      1000
weighted avg       1.00      0.91      0.95      1000



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
